# GuardianCGM: Module 06 - Drift Monitoring System Demo

**Author:** Alex Domingues Batista, PhD

**Objective:** Demonstrate the production drift monitoring system for continuous glucose monitoring (CGM) prediction models. This notebook shows how to detect statistical drift, performance degradation, and bias drift that would trigger model retraining alerts.

**Why this matters (Production MLOps Focus):**
- Most ML projects fail in production due to lack of monitoring and maintenance
- Medical devices require continuous performance validation (FDA 21 CFR Part 820, EU MDR PMCF)
- Model performance degrades over time due to sensor drift, population changes, and data quality issues
- Proactive monitoring prevents clinical safety incidents and regulatory non-compliance

**Key Learnings & Highlights:**
- Integration of drift monitoring system with trained models
- Simulation of realistic production scenarios (sensor degradation, population shift)
- Multi-level alerting system (INFO, WARNING, CRITICAL, RETRAINING_REQUIRED)
- Automated PMCF report generation for regulatory submissions
- Production-ready code architecture and logging

## 1. Setup and Import Drift Monitor

In [2]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import the drift monitoring module (rename to use as Python module)
import importlib.util
spec = importlib.util.spec_from_file_location("drift_monitoring", "05_Drift_Monitoring_Strategy.py")
drift_monitoring = importlib.util.module_from_spec(spec)
spec.loader.exec_module(drift_monitoring)

DriftMonitor = drift_monitoring.DriftMonitor
MonitoringThresholds = drift_monitoring.MonitoringThresholds

print("✓ Drift monitoring system imported successfully")

✓ Drift monitoring system imported successfully


## 2. Initialize Drift Monitor

The drift monitor requires:
- Path to trained model (from Module 02)
- Reference dataset (training data for comparison)
- Optional: Custom thresholds for alerting

In [3]:
# Initialize with default thresholds
monitor = DriftMonitor(
    model_path='models/glucose_rf_v1.pkl',
    reference_data_path='data/processed_biomarkers.csv',
    log_dir='logs',
    report_dir='reports'
)

print("\n✓ Drift monitor initialized")
print(f"  Model: {monitor.model_path.name}")
print(f"  Reference data samples: {len(monitor.reference_data)}")
print(f"  Features monitored: {len(monitor.feature_cols)}")

2026-01-30 11:26:58,603 | INFO | Model loaded: RandomForestRegressor
2026-01-30 11:26:58,611 | INFO | Reference data loaded: 846 samples
2026-01-30 11:26:58,612 | INFO | DriftMonitor initialized successfully
2026-01-30 11:26:58,613 | INFO | Model: models/glucose_rf_v1.pkl
2026-01-30 11:26:58,614 | INFO | Reference data: data/processed_biomarkers.csv



✓ Drift monitor initialized
  Model: glucose_rf_v1.pkl
  Reference data samples: 846
  Features monitored: 8


## 3. Scenario 1: Normal Operation (No Drift)

Test the monitoring system with a subset of the reference data to verify it passes all checks.

In [4]:
# Load reference data and take a sample for testing
reference_data = pd.read_csv('data/processed_biomarkers.csv')
normal_batch = reference_data.sample(n=200, random_state=42)

print("Testing Scenario 1: Normal Operation")
print(f"Batch size: {len(normal_batch)} samples\n")

# Run drift check
report_normal = monitor.check_drift(normal_batch)

# Display summary
print("\n" + "="*80)
print("SCENARIO 1 RESULTS")
print("="*80)
print(report_normal['summary'])

2026-01-30 11:27:01,823 | INFO | ================================================================================
2026-01-30 11:27:01,824 | INFO | DRIFT MONITORING CHECK INITIATED
2026-01-30 11:27:01,825 | INFO | New data batch: 200 samples
2026-01-30 11:27:01,826 | INFO | ================================================================================
2026-01-30 11:27:01,826 | INFO | 
[1/5] Running data quality checks...
2026-01-30 11:27:01,829 | INFO | ✓ Missing data rate: 0.0% (OK)
2026-01-30 11:27:01,832 | INFO | ✓ Outlier rate: 0.0% (OK)
2026-01-30 11:27:01,833 | INFO | ✓ All glucose values within physiological range (OK)
2026-01-30 11:27:01,834 | INFO | 
[2/5] Running statistical drift detection...
2026-01-30 11:27:01,838 | INFO | ✓ Feature 'glucose_raw': No significant drift (p=0.999)
2026-01-30 11:27:01,847 | INFO | ✓ Feature 'glucose_smooth': No significant drift (p=0.968)
2026-01-30 11:27:01,853 | INFO | ✓ Feature 'velocity': No significant drift (p=0.999)
2026-01-30 11:27:01

Testing Scenario 1: Normal Operation
Batch size: 200 samples


SCENARIO 1 RESULTS

⚠️  WARNINGS DETECTED
Total Alerts: 1 (Critical: 0, Warning: 1)
Samples Monitored: 200
Timestamp: 2026-01-30T11:27:01.826886
Retraining Required: False

Performance Metrics:
  - RMSE: 2.53 mg/dL
  - MAE: 1.44 mg/dL
  - Clarke Zone A+B: 100.0%



## 4. Scenario 2: Statistical Drift (Population Shift)

Simulate a scenario where the patient population changes (e.g., more elderly patients with higher glucose variability).

In [5]:
# Create drifted data by adding systematic bias to glucose features
drifted_batch = reference_data.sample(n=200, random_state=99).copy()

# Simulate sensor drift: increase glucose readings by 10% + add noise
for col in ['glucose_raw', 'glucose_smooth', 'lag_15m', 'lag_30m', 'lag_60m']:
    if col in drifted_batch.columns:
        drifted_batch[col] = drifted_batch[col] * 1.10 + np.random.normal(0, 5, len(drifted_batch))

# Adjust velocity and acceleration accordingly
if 'velocity' in drifted_batch.columns:
    drifted_batch['velocity'] = drifted_batch['velocity'] * 1.15
if 'acceleration' in drifted_batch.columns:
    drifted_batch['acceleration'] = drifted_batch['acceleration'] * 1.20

print("Testing Scenario 2: Statistical Drift (Population Shift)")
print(f"Batch size: {len(drifted_batch)} samples")
print("Simulated drift: +10% glucose shift + increased variability\n")

# Run drift check
report_drift = monitor.check_drift(drifted_batch)

# Display summary
print("\n" + "="*80)
print("SCENARIO 2 RESULTS")
print("="*80)
print(report_drift['summary'])

# Show detected drift alerts
drift_alerts = [a for a in report_drift['alerts'] if a.category == 'STATISTICAL_DRIFT']
if drift_alerts:
    print(f"\nStatistical Drift Alerts Detected: {len(drift_alerts)}")
    for alert in drift_alerts[:3]:  # Show first 3
        print(f"  - {alert.message}")

2026-01-30 11:27:04,736 | INFO | ================================================================================
2026-01-30 11:27:04,737 | INFO | DRIFT MONITORING CHECK INITIATED
2026-01-30 11:27:04,739 | INFO | New data batch: 200 samples
2026-01-30 11:27:04,740 | INFO | ================================================================================
2026-01-30 11:27:04,741 | INFO | 
[1/5] Running data quality checks...
2026-01-30 11:27:04,745 | INFO | ✓ Missing data rate: 0.0% (OK)
2026-01-30 11:27:04,755 | INFO | ✓ Outlier rate: 0.0% (OK)
2026-01-30 11:27:04,757 | INFO | ✓ All glucose values within physiological range (OK)
2026-01-30 11:27:04,757 | INFO | 
[2/5] Running statistical drift detection...
2026-01-30 11:27:04,761 | WARNING | Significant distribution drift detected in 'glucose_raw' (KS p-value: 0.0000)
2026-01-30 11:27:04,769 | ERROR | Critical PSI for 'glucose_raw': 0.304 (threshold: 0.25)
2026-01-30 11:27:04,772 | WARNING | Significant distribution drift detected in 'gl

Testing Scenario 2: Statistical Drift (Population Shift)
Batch size: 200 samples
Simulated drift: +10% glucose shift + increased variability


SCENARIO 2 RESULTS

🚨 RETRAINING REQUIRED
Total Alerts: 13 (Critical: 11, Warning: 2)
Samples Monitored: 200
Timestamp: 2026-01-30T11:27:04.741068
Retraining Required: True

Performance Metrics:
  - RMSE: 12.95 mg/dL
  - MAE: 10.88 mg/dL
  - Clarke Zone A+B: 100.0%


Statistical Drift Alerts Detected: 12
  - Significant distribution drift detected in 'glucose_raw' (KS p-value: 0.0000)
  - Critical PSI for 'glucose_raw': 0.304 (threshold: 0.25)
  - Significant distribution drift detected in 'glucose_smooth' (KS p-value: 0.0000)


## 5. Scenario 3: Performance Degradation (Sensor Failure)

Simulate sensor degradation that causes prediction accuracy to drop below acceptable thresholds.

In [6]:
# Create degraded data with high noise
degraded_batch = reference_data.sample(n=200, random_state=123).copy()

# Simulate severe sensor degradation: add large random noise
for col in monitor.feature_cols:
    if col in degraded_batch.columns:
        noise_std = degraded_batch[col].std() * 0.8  # 80% of original std as noise
        degraded_batch[col] = degraded_batch[col] + np.random.normal(0, noise_std, len(degraded_batch))

print("Testing Scenario 3: Performance Degradation (Sensor Failure)")
print(f"Batch size: {len(degraded_batch)} samples")
print("Simulated issue: High sensor noise (80% std added to features)\n")

# Run drift check
report_degraded = monitor.check_drift(degraded_batch)

# Display summary
print("\n" + "="*80)
print("SCENARIO 3 RESULTS")
print("="*80)
print(report_degraded['summary'])

# Show performance alerts
perf_alerts = [a for a in report_degraded['alerts'] if a.category == 'PERFORMANCE']
if perf_alerts:
    print(f"\nPerformance Alerts Detected: {len(perf_alerts)}")
    for alert in perf_alerts:
        print(f"  [{alert.severity}] {alert.message}")

2026-01-30 11:27:08,105 | INFO | ================================================================================
2026-01-30 11:27:08,105 | INFO | DRIFT MONITORING CHECK INITIATED
2026-01-30 11:27:08,106 | INFO | New data batch: 200 samples
2026-01-30 11:27:08,107 | INFO | ================================================================================
2026-01-30 11:27:08,108 | INFO | 
[1/5] Running data quality checks...
2026-01-30 11:27:08,110 | INFO | ✓ Missing data rate: 0.0% (OK)
2026-01-30 11:27:08,114 | INFO | ✓ Outlier rate: 0.0% (OK)
2026-01-30 11:27:08,115 | ERROR | 3 samples outside physiological glucose range (40-400 mg/dL)
2026-01-30 11:27:08,116 | INFO | 
[2/5] Running statistical drift detection...
2026-01-30 11:27:08,119 | INFO | ✓ Feature 'glucose_raw': No significant drift (p=0.075)
2026-01-30 11:27:08,126 | WARNING | Warning PSI for 'glucose_raw': 0.169 (threshold: 0.1)
2026-01-30 11:27:08,128 | INFO | ✓ Feature 'glucose_smooth': No significant drift (p=0.068)
2026-0

Testing Scenario 3: Performance Degradation (Sensor Failure)
Batch size: 200 samples
Simulated issue: High sensor noise (80% std added to features)


SCENARIO 3 RESULTS

🚨 RETRAINING REQUIRED
Total Alerts: 11 (Critical: 2, Warning: 9)
Samples Monitored: 200
Timestamp: 2026-01-30T11:27:08.108127
Retraining Required: True

Performance Metrics:
  - RMSE: 19.80 mg/dL
  - MAE: 15.38 mg/dL
  - Clarke Zone A+B: 99.5%


Performance Alerts Detected: 1
  [CRITICAL] CRITICAL: RMSE 19.80 mg/dL exceeds threshold 9.0 mg/dL


## 6. Scenario 4: Data Quality Issues

Simulate missing data and outliers that indicate integration or sensor issues.

In [7]:
# Create batch with data quality issues
quality_batch = reference_data.sample(n=200, random_state=456).copy()

# Introduce missing values (10% of data)
for col in monitor.feature_cols[:4]:  # Affect first 4 features
    if col in quality_batch.columns:
        missing_idx = np.random.choice(quality_batch.index, size=int(len(quality_batch) * 0.10), replace=False)
        quality_batch.loc[missing_idx, col] = np.nan

# Introduce extreme outliers (5% of data)
if 'glucose_raw' in quality_batch.columns:
    outlier_idx = np.random.choice(quality_batch.index, size=int(len(quality_batch) * 0.05), replace=False)
    quality_batch.loc[outlier_idx, 'glucose_raw'] = np.random.choice([30, 450], size=len(outlier_idx))  # Out of range

print("Testing Scenario 4: Data Quality Issues")
print(f"Batch size: {len(quality_batch)} samples")
print("Simulated issues: 10% missing values + 5% out-of-range outliers\n")

# Run drift check
report_quality = monitor.check_drift(quality_batch)

# Display summary
print("\n" + "="*80)
print("SCENARIO 4 RESULTS")
print("="*80)
print(report_quality['summary'])

# Show data quality alerts
quality_alerts = [a for a in report_quality['alerts'] if a.category == 'DATA_QUALITY']
if quality_alerts:
    print(f"\nData Quality Alerts Detected: {len(quality_alerts)}")
    for alert in quality_alerts:
        print(f"  [{alert.severity}] {alert.message}")

2026-01-30 11:27:13,345 | INFO | ================================================================================
2026-01-30 11:27:13,346 | INFO | DRIFT MONITORING CHECK INITIATED
2026-01-30 11:27:13,347 | INFO | New data batch: 200 samples
2026-01-30 11:27:13,347 | INFO | ================================================================================
2026-01-30 11:27:13,348 | INFO | 
[1/5] Running data quality checks...
2026-01-30 11:27:13,350 | INFO | ✓ Missing data rate: 5.0% (OK)
2026-01-30 11:27:13,355 | INFO | ✓ Outlier rate: 0.5% (OK)
2026-01-30 11:27:13,356 | ERROR | 10 samples outside physiological glucose range (40-400 mg/dL)
2026-01-30 11:27:13,357 | INFO | 
[2/5] Running statistical drift detection...
2026-01-30 11:27:13,360 | INFO | ✓ Feature 'glucose_raw': No significant drift (p=0.914)
2026-01-30 11:27:13,368 | INFO | ✓ Feature 'glucose_smooth': No significant drift (p=0.854)
2026-01-30 11:27:13,375 | INFO | ✓ Feature 'velocity': No significant drift (p=0.665)
2026-01-3

Testing Scenario 4: Data Quality Issues
Batch size: 200 samples
Simulated issues: 10% missing values + 5% out-of-range outliers


SCENARIO 4 RESULTS

🚨 RETRAINING REQUIRED
Total Alerts: 2 (Critical: 2, Warning: 0)
Samples Monitored: 200
Timestamp: 2026-01-30T11:27:13.348410
Retraining Required: True

Performance Metrics:
  - RMSE: 9.15 mg/dL
  - MAE: 4.19 mg/dL
  - Clarke Zone A+B: 100.0%


Data Quality Alerts Detected: 1
  [CRITICAL] 10 samples outside physiological glucose range (40-400 mg/dL)


## 7. Alert Summary Across All Scenarios

In [8]:
# Compile results from all scenarios
scenarios = [
    ('Normal Operation', report_normal),
    ('Statistical Drift', report_drift),
    ('Performance Degradation', report_degraded),
    ('Data Quality Issues', report_quality)
]

summary_df = pd.DataFrame([
    {
        'Scenario': name,
        'Total Alerts': len(report['alerts']),
        'Critical': sum(1 for a in report['alerts'] if a.severity == 'CRITICAL'),
        'Warning': sum(1 for a in report['alerts'] if a.severity == 'WARNING'),
        'Retraining Required': 'YES' if report['retraining_required'] else 'NO',
        'RMSE (mg/dL)': report['metrics'].get('performance', {}).get('rmse', 'N/A')
    }
    for name, report in scenarios
])

print("\n" + "="*80)
print("COMPREHENSIVE SCENARIO COMPARISON")
print("="*80)
print(summary_df.to_string(index=False))

# Retraining trigger analysis
retraining_needed = sum(1 for _, report in scenarios if report['retraining_required'])
print(f"\n📊 Summary: {retraining_needed}/{len(scenarios)} scenarios triggered retraining alerts")


COMPREHENSIVE SCENARIO COMPARISON
               Scenario  Total Alerts  Critical  Warning Retraining Required  RMSE (mg/dL)
       Normal Operation             1         0        1                  NO      2.532278
      Statistical Drift            13        11        2                 YES     12.953780
Performance Degradation            11         2        9                 YES     19.800692
    Data Quality Issues             2         2        0                 YES      9.154669

📊 Summary: 3/4 scenarios triggered retraining alerts


## 8. Generate Post-Market Clinical Follow-up (PMCF) Report

Regulatory authorities require periodic surveillance reports. The drift monitor can automatically generate these.

In [9]:
# Generate PMCF report for all monitored data
pmcf_report_path = monitor.generate_pmcf_report(
    output_path='reports/PMCF_Demo_Report.txt'
)

print(f"✓ PMCF Report Generated: {pmcf_report_path}\n")

# Display report preview
with open(pmcf_report_path, 'r') as f:
    report_content = f.read()
    print("Report Preview (first 50 lines):")
    print("="*80)
    print('\n'.join(report_content.split('\n')[:50]))
    print("\n... [Report continues] ...")

2026-01-30 11:28:47,756 | INFO | PMCF report generated: reports/PMCF_Demo_Report.txt


✓ PMCF Report Generated: reports/PMCF_Demo_Report.txt

Report Preview (first 50 lines):

POST-MARKET CLINICAL FOLLOW-UP (PMCF) REPORT
GuardianCGM Glucose Prediction Model

Report Generated: 2026-01-30 11:28:47
Reporting Period: Inception to Present

1. EXECUTIVE SUMMARY
-------------------
Total Monitoring Events: 27
Critical Alerts: 15
Warning Alerts: 12
Retraining Events: 0

2. ALERT BREAKDOWN BY CATEGORY
-------------------------------

STATISTICAL_DRIFT:
  Total: 22
  Critical: 10
  Warnings: 12

PERFORMANCE:
  Total: 3
  Critical: 3
  Warnings: 0

DATA_QUALITY:
  Total: 2
  Critical: 2
  Warnings: 0


3. REGULATORY COMPLIANCE STATUS
--------------------------------
FDA 21 CFR Part 820 (Quality System): [COMPLIANT/NON-COMPLIANT]
EU MDR (2017/745) PMCF: [COMPLIANT/NON-COMPLIANT]
ISO 13485: [COMPLIANT/NON-COMPLIANT]

4. CORRECTIVE ACTIONS TAKEN
----------------------------
[To be filled by clinical/regulatory team]

5. RISK ASSESSMENT
------------------
Current Risk Level: [LOW/MEDIU

## 9. Production Integration Example

Show how this would be integrated into a production monitoring pipeline.

In [10]:
# Example production monitoring function
def production_monitoring_pipeline(new_data_path: str):
    """
    Example production monitoring workflow.
    Would be triggered daily/weekly by scheduler (cron, Airflow, etc.)
    """
    print("🔄 Production Monitoring Pipeline Started")
    print("="*80)
    
    # 1. Load new production data
    print("[1/4] Loading production data...")
    new_data = pd.read_csv(new_data_path)
    print(f"      ✓ Loaded {len(new_data)} samples")
    
    # 2. Run drift monitoring
    print("[2/4] Running drift detection...")
    report = monitor.check_drift(new_data)
    print(f"      ✓ Detected {len(report['alerts'])} alerts")
    
    # 3. Check retraining trigger
    print("[3/4] Evaluating retraining necessity...")
    if report['retraining_required']:
        print("      🚨 RETRAINING REQUIRED - Triggering ML pipeline")
        # In production: trigger_retraining_job()
        # Send alerts to data science team
    else:
        print("      ✓ Model performance acceptable")
    
    # 4. Send alerts to monitoring dashboard
    print("[4/4] Sending alerts to monitoring system...")
    critical_alerts = [a for a in report['alerts'] if a.severity == 'CRITICAL']
    if critical_alerts:
        print(f"      ⚠️  {len(critical_alerts)} critical alerts sent to PagerDuty/Slack")
        # In production: send_alerts_to_slack(critical_alerts)
    else:
        print("      ✓ No critical alerts")
    
    print("\n" + "="*80)
    print("✓ Pipeline completed successfully")
    return report

# Simulate running the pipeline
print("\nSimulating Production Monitoring Pipeline:\n")
pipeline_report = production_monitoring_pipeline('data/processed_biomarkers.csv')

2026-01-30 11:28:50,432 | INFO | ================================================================================
2026-01-30 11:28:50,432 | INFO | DRIFT MONITORING CHECK INITIATED
2026-01-30 11:28:50,433 | INFO | New data batch: 846 samples
2026-01-30 11:28:50,434 | INFO | ================================================================================
2026-01-30 11:28:50,435 | INFO | 
[1/5] Running data quality checks...
2026-01-30 11:28:50,438 | INFO | ✓ Missing data rate: 0.0% (OK)
2026-01-30 11:28:50,446 | INFO | ✓ Outlier rate: 0.0% (OK)
2026-01-30 11:28:50,447 | INFO | ✓ All glucose values within physiological range (OK)
2026-01-30 11:28:50,448 | INFO | 
[2/5] Running statistical drift detection...
2026-01-30 11:28:50,450 | INFO | ✓ Feature 'glucose_raw': No significant drift (p=1.000)
2026-01-30 11:28:50,458 | INFO | ✓ Feature 'glucose_smooth': No significant drift (p=1.000)
2026-01-30 11:28:50,466 | INFO | ✓ Feature 'velocity': No significant drift (p=1.000)
2026-01-30 11:28:50


Simulating Production Monitoring Pipeline:

🔄 Production Monitoring Pipeline Started
[1/4] Loading production data...
      ✓ Loaded 846 samples
[2/4] Running drift detection...
      ✓ Detected 0 alerts
[3/4] Evaluating retraining necessity...
      ✓ Model performance acceptable
[4/4] Sending alerts to monitoring system...
      ✓ No critical alerts

✓ Pipeline completed successfully


## 10. Monitoring Threshold Customization

Demonstrate how to customize thresholds for different clinical contexts or regulatory requirements.

In [11]:
# Create custom thresholds for stricter monitoring
strict_thresholds = MonitoringThresholds(
    rmse_warning=6.0,  # Lower threshold (stricter)
    rmse_critical=7.5,
    clarke_ab_min=97.0,  # Higher FDA threshold (stricter)
    psi_warning=0.08,  # Lower PSI tolerance
    psi_critical=0.15
)

# Initialize monitor with custom thresholds
strict_monitor = DriftMonitor(
    model_path='models/glucose_rf_v1.pkl',
    reference_data_path='data/processed_biomarkers.csv',
    thresholds=strict_thresholds,
    log_dir='logs',
    report_dir='reports'
)

print("Custom Threshold Configuration:")
print("="*80)
print(f"RMSE Warning: {strict_thresholds.rmse_warning} mg/dL (default: 7.0)")
print(f"RMSE Critical: {strict_thresholds.rmse_critical} mg/dL (default: 9.0)")
print(f"Clarke A+B Min: {strict_thresholds.clarke_ab_min}% (default: 95.0%)")
print(f"PSI Warning: {strict_thresholds.psi_warning} (default: 0.10)")
print(f"PSI Critical: {strict_thresholds.psi_critical} (default: 0.25)")

# Test with stricter thresholds
print("\nTesting with stricter thresholds...\n")
strict_report = strict_monitor.check_drift(normal_batch)

print("\nComparison:")
print(f"  Standard thresholds: {len(report_normal['alerts'])} alerts")
print(f"  Strict thresholds: {len(strict_report['alerts'])} alerts")
print(f"\n  Retraining required (standard): {report_normal['retraining_required']}")
print(f"  Retraining required (strict): {strict_report['retraining_required']}")

2026-01-30 11:28:53,423 | INFO | Model loaded: RandomForestRegressor
2026-01-30 11:28:53,428 | INFO | Reference data loaded: 846 samples
2026-01-30 11:28:53,429 | INFO | DriftMonitor initialized successfully
2026-01-30 11:28:53,429 | INFO | Model: models/glucose_rf_v1.pkl
2026-01-30 11:28:53,430 | INFO | Reference data: data/processed_biomarkers.csv
2026-01-30 11:28:53,431 | INFO | ================================================================================
2026-01-30 11:28:53,432 | INFO | DRIFT MONITORING CHECK INITIATED
2026-01-30 11:28:53,433 | INFO | New data batch: 200 samples
2026-01-30 11:28:53,434 | INFO | ================================================================================
2026-01-30 11:28:53,435 | INFO | 
[1/5] Running data quality checks...
2026-01-30 11:28:53,438 | INFO | ✓ Missing data rate: 0.0% (OK)
2026-01-30 11:28:53,442 | INFO | ✓ Outlier rate: 0.0% (OK)
2026-01-30 11:28:53,443 | INFO | ✓ All glucose values within physiological range (OK)
2026-01-30 11

Custom Threshold Configuration:
RMSE Warning: 6.0 mg/dL (default: 7.0)
RMSE Critical: 7.5 mg/dL (default: 9.0)
Clarke A+B Min: 97.0% (default: 95.0%)
PSI Warning: 0.08 (default: 0.10)
PSI Critical: 0.15 (default: 0.25)

Testing with stricter thresholds...


Comparison:
  Standard thresholds: 1 alerts
  Strict thresholds: 1 alerts

  Retraining required (standard): False
  Retraining required (strict): False


## 11. Conclusion & Production Recommendations

### **Key Takeaways:**

1. **Proactive Monitoring is Essential**: The drift monitoring system detected issues across all failure scenarios before they compromised clinical safety

2. **Multi-Dimensional Detection**: Statistical drift, performance degradation, bias drift, and data quality issues require different detection methods—no single metric is sufficient

3. **Regulatory Compliance**: Automated PMCF reporting and audit logging satisfy FDA 21 CFR Part 820 and EU MDR requirements

4. **Actionable Alerts**: Multi-level severity (INFO, WARNING, CRITICAL, RETRAINING_REQUIRED) enables appropriate response without alert fatigue

5. **Customizable Thresholds**: Different clinical contexts (pediatric vs. adult, Type 1 vs. Type 2 diabetes) may require adjusted monitoring sensitivity

### **Production Deployment Recommendations:**

#### **Infrastructure:**
- Deploy as containerized microservice (Docker) for horizontal scaling
- Integrate with message queue (Kafka/RabbitMQ) for real-time data streaming
- Use time-series database (InfluxDB/TimescaleDB) for metric trending
- Connect to monitoring dashboards (Grafana/Datadog) for visualization

#### **Alerting:**
- Route CRITICAL alerts to PagerDuty for immediate on-call response
- Send WARNING alerts to Slack channels for data science team awareness
- Generate weekly summary reports for stakeholders
- Store all alerts in searchable database for audit trails

#### **Automation:**
- Schedule daily drift checks via Airflow/Prefect
- Trigger automated retraining pipelines when thresholds exceeded
- Generate monthly PMCF reports for regulatory submissions
- Archive historical data for longitudinal analysis

#### **Governance:**
- Establish change control process for threshold modifications
- Document all monitoring decisions in risk management files (ISO 14971)
- Conduct quarterly reviews of alert patterns and false positive rates
- Maintain version control for drift detection configurations

### **Next Steps:**
1. Validate monitoring system on real-world CGM data (Dexcom, Abbott FreeStyle Libre)
2. Establish baseline drift patterns during pilot deployment
3. Calibrate alert thresholds based on clinical feedback
4. Integrate with existing hospital IT systems (EPIC, Cerner)
5. Conduct prospective validation study for regulatory approval

---

> _"In production medical AI, the model is just the beginning—continuous monitoring and maintenance are what keep patients safe."_

**Alex Domingues Batista, PhD**  
*Data Scientist*